# Track Data Enriching

**Main objective:** include the `main genre` of the *artist* and `album image URI` in the dataset, expanding the analysis possibilities. Also, the `platform` column is normalized, and the complete information removed from the dataset for privacy and security reasons.

To do the *data enriching* steps, I used the [Spotipy](https://github.com/spotipy-dev/spotipy) library to make the interface with [Spotify's Web API](https://developer.spotify.com/documentation/web-api).

Regarding the data, I used the **Extended streaming history** download, that can be found under *Privacy* settings on your Spotify account. For more information on how to get yours, here is a [simple tutorial from Quora](https://www.quora.com/How-can-I-download-my-Spotify-data).

*Disclaimer*: this is my own data, thus, my own music taste and Spotify usage. I am not responsible if your musical taste is not as good as mine 😂

## Imports and inicializations

In [1]:
import pandas as pd
import spotipy
import time

from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
sp

In [4]:
tracks_history = pd.read_csv('consolidated_spotify_data_2024.csv')
tracks_history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189780 entries, 0 to 189779
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   timestamp_play     189780 non-null  object
 1   platform           189780 non-null  object
 2   ms_played          189780 non-null  int64 
 3   track_name         189780 non-null  object
 4   artist_name        189780 non-null  object
 5   album_name         189780 non-null  object
 6   spotify_track_uri  189780 non-null  object
 7   reason_start       189713 non-null  object
 8   reason_end         151097 non-null  object
dtypes: int64(1), object(8)
memory usage: 13.0+ MB


In [47]:
tracks_history.head(5)

,timestamp_play,platform,ms_played,track_name,artist_name,album_name,spotify_track_uri,reason_start,reason_end
0,2022-01-22T12:59:36Z,"Android OS 11 API 30 (Xiaomi, M2101K6G)",2567,GALOPA,PEDRO SAMPAIO,GALOPA,spotify:track:2wG1R0uDFwyobcWzVssC1J,clickrow,endplay
1,2022-01-22T13:00:24Z,"Android OS 11 API 30 (Xiaomi, M2101K6G)",47653,A QUEDA,Gloria Groove,A QUEDA,spotify:track:2s9BO8c0co0PmgBiUoTT17,clickrow,endplay
2,2022-01-22T13:04:16Z,"Android OS 11 API 30 (Xiaomi, M2101K6G)",218223,Convite de Casamento,Luan Santana,Confraternização Família Santana 2,spotify:track:3BDnZJC9yaj01jtqpyDYzG,clickrow,endplay
3,2022-01-22T13:05:17Z,"Android OS 11 API 30 (Xiaomi, M2101K6G)",61025,Acordando o Prédio,Luan Santana,Acordando o Prédio,spotify:track:40bK2uosUmAS92c17n98xd,clickrow,endplay
4,2022-01-22T13:05:41Z,"Android OS 11 API 30 (Xiaomi, M2101K6G)",23375,Juntos,Paula Fernandes,Juntos,spotify:track:2PIWldP9oUhb9iqe3EJh1e,clickrow,endplay


## Data Enriching

There is some interesting facts about this part, as I am filling the notebook after completing the development:

I found out, in the worst way, that APIs have *request rate limits*. This is, on the first try, I sent around 4k requests to **Spotify's API**, what is impractical and could get my API access cut (luckily, this did not happen).

**Datasets are unique**, so knowing its *structure and context* is fundamental. This is a **expanded tracks history**, and knowing my own musical habits, I know that most tracks here were played hundreds of times during the last 10 years (focus on Linkin Park and Green Day). It is, instead of sending a request for each track in the list (182k) multiple times, I can unify them and send only one request per track. Of course, it probably is still a big number, but already better than the initial one.

In [5]:
unique_artists = tracks_history.groupby('artist_name')[['spotify_track_uri']].first().reset_index()
print('Artists:', len(unique_artists))
unique_artists.sample(10)

Artists: 9480


,artist_name,spotify_track_uri
7372,Silversage,spotify:track:5t8uAGGdxMMnHksPaDzTLb
1260,C Y G N,spotify:track:0XTRycTWH8TR2xBEjwByNS
174,Adriatique,spotify:track:01NYi0fg9hHQZO8VQbDQL2
5694,Mrozu,spotify:track:6xOZxa5aOEdLvXSTMwDCjm
985,Blasterjaxx,spotify:track:3UdCu8rkzx25I7CVpEE4ga
9326,jhfly,spotify:track:5hgX8pSZSOcKhsvzldsgWy
1381,Caroline Phillips,spotify:track:40Sa0LZCp1p1X2tGh47hrs
1108,Bowling For Soup,spotify:track:2monU2EJfR5m53ciaHNE6z
6244,Pablito Pesadilla,spotify:track:0kTMK4gNFfLXaTb62w1UaJ
9237,brakence,spotify:track:7cELheBXF3jIShGuewCy44


Now, instead of sending a request for each track as I first tried with the [Get Track endpoint](https://developer.spotify.com/documentation/web-api/reference/get-track), I discovered I can actually send a batch of `track_id` and [Get Several Tracks](https://developer.spotify.com/documentation/web-api/reference/get-several-tracks) at once. *Isn't this beautiful?*

[Spotify's documentation](https://developer.spotify.com/documentation/web-api/reference/get-several-tracks) says it has a limit of 100 IDs per request. But, [Spotipy's documentation](https://spotipy.readthedocs.io/en/2.16.1/#spotipy.client.Spotify.tracks) says it has a 50 IDs limitation. I tried both, but only 50 worked, so let's stay under this limit.

Now, having the `batch_size` defined, it was needed to create the batches for further usage.

In [6]:
unique_tracks = tracks_history[['spotify_track_uri']].drop_duplicates()
unique_tracks_ids = unique_tracks['spotify_track_uri'].tolist()

# Spotipy's documentation says the limit is 50 IDs
batch_size = 50

track_uris_batches = []
for i in range(0, len(unique_tracks_ids), batch_size):
    track_uris_batches.append(unique_tracks_ids[i:i+batch_size])

print('Batches:', len(track_uris_batches))

Batches: 690


Here is the logic to get the information needed. First, we take each *batch* and make an API request. With the *response tracks* in hand, we can go through each track and then extract the information intended.

1. **Album cover URI:** this is already included in the *track* JSON. As Spotify's response include several URIs related to different sizes and qualities, we just want to get the first one, that should be the smallest one available (if included).

2. **Artist:** unfortunately, the *artist's genre* is not included in the *track* JSON. We only have access to *artist's ID* here, so this will be used later to make another request.

Just as I mentioned before, Spotify's API has a rate limit based on a 30-seconds window. I was not able to find exactly how much is this limit, but I thought it was a good idea to keep my requests under 4 per second (or one each 250ms). That is why I added a `request_delay` variable that controls the `time.sleep` function after each request.

In [18]:
tracks_enriched = []
counter = 0
request_delay = 0.25 # Delay (in seconds) between requests to avoid API rate limitation

for batch in track_uris_batches:
    tracks = sp.tracks(batch)['tracks']
    
    for track in tracks:
        artist_id = track['artists'][0]['id']
        album = track['album']

        # Get album cover URL from the smallest available image size
        album_cover_uri = album['images'][0]['url'] if album['images'] else None
        
        tracks_enriched.append({
            'spotify_track_uri': track['uri'],
            'track_id': track['id'],
            'artist_id': artist_id,
            'album_cover_uri': album_cover_uri
        })
        
    # Counting and printing for progress tracking
    counter += 1
    
    if counter % 10 == 0:
        print(f'Progress: {counter} out of {len(track_uris_batches)} ({int(counter / len(track_uris_batches) * 100)}%)')
    
    time.sleep(request_delay)

print(f'Processed a total of {counter} batches.')

Progress: 10 out of 690 (1%)
Progress: 20 out of 690 (2%)
Progress: 30 out of 690 (4%)
Progress: 40 out of 690 (5%)
Progress: 50 out of 690 (7%)
Progress: 60 out of 690 (8%)
Progress: 70 out of 690 (10%)
Progress: 80 out of 690 (11%)
Progress: 90 out of 690 (13%)
Progress: 100 out of 690 (14%)
Progress: 110 out of 690 (15%)
Progress: 120 out of 690 (17%)
Progress: 130 out of 690 (18%)
Progress: 140 out of 690 (20%)
Progress: 150 out of 690 (21%)
Progress: 160 out of 690 (23%)
Progress: 170 out of 690 (24%)
Progress: 180 out of 690 (26%)
Progress: 190 out of 690 (27%)
Progress: 200 out of 690 (28%)
Progress: 210 out of 690 (30%)
Progress: 220 out of 690 (31%)
Progress: 230 out of 690 (33%)
Progress: 240 out of 690 (34%)
Progress: 250 out of 690 (36%)
Progress: 260 out of 690 (37%)
Progress: 270 out of 690 (39%)
Progress: 280 out of 690 (40%)
Progress: 290 out of 690 (42%)
Progress: 300 out of 690 (43%)
Progress: 310 out of 690 (44%)
Progress: 320 out of 690 (46%)
Progress: 330 out of 69

In [19]:
df_tracks_enriched = pd.DataFrame(tracks_enriched)
df_tracks_enriched

,spotify_track_uri,track_id,artist_id,album_cover_uri
0,spotify:track:2wG1R0uDFwyobcWzVssC1J,2wG1R0uDFwyobcWzVssC1J,5wbf52LA6kcaboHSN6NEF1,https://i.scdn.co/image/ab67616d0000b27380dadd...
1,spotify:track:2s9BO8c0co0PmgBiUoTT17,2s9BO8c0co0PmgBiUoTT17,7rXMvXRnWHaSwnVvPeUUfw,https://i.scdn.co/image/ab67616d0000b273ea1781...
2,spotify:track:3BDnZJC9yaj01jtqpyDYzG,3BDnZJC9yaj01jtqpyDYzG,3qvcCP2J0fWi0m0uQDUf6r,https://i.scdn.co/image/ab67616d0000b273e70918...
3,spotify:track:40bK2uosUmAS92c17n98xd,40bK2uosUmAS92c17n98xd,3qvcCP2J0fWi0m0uQDUf6r,https://i.scdn.co/image/ab67616d0000b273d393a0...
4,spotify:track:2PIWldP9oUhb9iqe3EJh1e,2PIWldP9oUhb9iqe3EJh1e,1nca3OA1kKCpP6aPJcBL92,https://i.scdn.co/image/ab67616d0000b2733c7178...
...,...,...,...,...
34484,spotify:track:1n108NGnovKSwv2z0J9Vmw,1n108NGnovKSwv2z0J9Vmw,53XhwfbYqKCa1cC15pYq2q,https://i.scdn.co/image/ab67616d0000b273dee648...
34485,spotify:track:5QbWc5ipIn02XePjT0Tvaf,5QbWc5ipIn02XePjT0Tvaf,7gOdHgIoIKoe4i9Tta6qdD,https://i.scdn.co/image/ab67616d0000b27338f63e...
34486,spotify:track:1xHexNjWpStUnoFCqNLanp,1xHexNjWpStUnoFCqNLanp,3h7RaVXBvdSNa7LXQtVYqH,https://i.scdn.co/image/ab67616d0000b2737027fe...
34487,spotify:track:2bQ9UIx9KQkOybSpuTHaL8,2bQ9UIx9KQkOybSpuTHaL8,5Pwc4xIPtQLFEnJriah9YJ,https://i.scdn.co/image/ab67616d0000b2734c4052...


Ok, so the last enriching step is to add the *artist's genre* on the dataset.

Considering that I already have every `artist_id` from the dataset on the *tracks* DataFrame, I just need to unify them exactly the same way as I did with the tracks.

For the same reason, it is natural to think that most part of my musical history is based on a small set of artists/bands, and just a few artists are present only once on the dataset (probably from LoFi playlists with weird names).

The same logic is applied here, as we also have the [Get Several Artists](https://developer.spotify.com/documentation/web-api/reference/get-multiple-artists) endpoint, so I can group them all in batches and then make the request.

In [20]:
artists_unique = df_tracks_enriched['artist_id'].drop_duplicates()
unique_artist_ids = artists_unique.tolist()

artist_id_batches = []
for i in range(0, len(unique_artist_ids), batch_size):
    artist_id_batches.append(unique_artist_ids[i:i+batch_size])

print('Batches:', len(artist_id_batches))

Batches: 189


The `genre` field from Spotify can have multiple values, for instance, one band can be considered as *Hard Rock* and *Classic Rock* at once, like AC/DC. For the extent of this work, I will only consider the first **genre** available as the "most important" one.

This can be polemic, as one artist/band can be known for a certain genre for most of people, but Spotify can put the first genre anything else. I don't exactly know at this point how much this will impact on the analysis, but I will leave this simple solution and see what happens later.

In [21]:
artists_enriched = []
counter = 0
request_delay = 0.25 # Delay (in seconds) between requests to avoid API rate limitation

for batch in artist_id_batches:
    artists = sp.artists(batch)['artists']
    
    for artist in artists:

        # Check if there are genres (handle cases where it might be empty)
        artist_genre = artist['genres'][0] if artist['genres'] else None
        
        artists_enriched.append({
            'artist_id': artist['id'],
            'artist_genre': artist_genre
        })
        
    # Counting and printing for progress tracking
    counter += 1
    
    if counter % 10 == 0:
        print(f'Progress: {counter} out of {len(artist_id_batches)} ({int(counter / len(artist_id_batches) * 100)}%)')
    
    time.sleep(request_delay)

print(f'Processed a total of {counter} batches.')

Progress: 10 out of 189 (5%)
Progress: 20 out of 189 (10%)
Progress: 30 out of 189 (15%)
Progress: 40 out of 189 (21%)
Progress: 50 out of 189 (26%)
Progress: 60 out of 189 (31%)
Progress: 70 out of 189 (37%)
Progress: 80 out of 189 (42%)
Progress: 90 out of 189 (47%)
Progress: 100 out of 189 (52%)
Progress: 110 out of 189 (58%)
Progress: 120 out of 189 (63%)
Progress: 130 out of 189 (68%)
Progress: 140 out of 189 (74%)
Progress: 150 out of 189 (79%)
Progress: 160 out of 189 (84%)
Progress: 170 out of 189 (89%)
Progress: 180 out of 189 (95%)
Processed a total of 189 batches.


In [24]:
df_artists_enriched = pd.DataFrame(artists_enriched)
df_artists_enriched.sample(10)

,artist_id,artist_genre
8375,4FUZzDnu4gBue46G99hesO,None
7878,2BVYdY4PyfCF9z4NrkhEB2,classic soul
5587,61R2K577GF9XaGmrzJIebF,lds youth
1266,3MZsBdqDrRTJihTHQrO6Dq,viral pop
2736,3wKV11EdqfY7lFaUeDUEC1,lo-fi beats
6448,3xgj17ZsWxxU86S4qlWvOi,new romantic
6557,49PC2thu4PsvOGs77fgXwA,background music
5384,11TplWqOPQBTmg2eiSLt1m,italian adult pop
125,6xlRSRMLgZbsSNd0BMobwy,funk carioca
8349,2WLu3mkdi8l6Jjm6HCVC9h,big room


Having both DataFrames already done with the intended fields added, *now we are two **JOINs** away from the objective*.

In [29]:
df_enriched = df_tracks_enriched.merge(
    df_artists_enriched,
    how='left',
    on='artist_id'
)

df_enriched.sample(10)

,spotify_track_uri,track_id,artist_id,album_cover_uri,artist_genre
1808,spotify:track:4KacUpvbA3Mfo05gttTjhN,4KacUpvbA3Mfo05gttTjhN,0qT79UgT5tY4yudH9VfsdT,https://i.scdn.co/image/ab67616d0000b2739a4821...,alternative metal
29536,spotify:track:6nLEcwqpVNOGRGtnV2pElc,6nLEcwqpVNOGRGtnV2pElc,2kgvaYV2tbNyOonFI3YlfH,https://i.scdn.co/image/ab67616d0000b273bfbe91...,slap house
25187,spotify:track:62I2gpsdePiOFaeGbqsfiM,62I2gpsdePiOFaeGbqsfiM,2XddLUPFBmTonCFy8uB3uc,https://i.scdn.co/image/ab67616d0000b2738df36a...,birmingham indie
12458,spotify:track:5lEgVd1KFeYiWm62zNXvjl,5lEgVd1KFeYiWm62zNXvjl,6SjLkpJ7cGqX5HWh23e74K,https://i.scdn.co/image/ab67616d0000b2736a09df...,funk carioca
6334,spotify:track:4rXYEn4CgmYL5UPSKY5CWf,4rXYEn4CgmYL5UPSKY5CWf,1Fx1sB95ek2EzaohffoEyM,https://i.scdn.co/image/ab67616d0000b2732b4432...,None
20892,spotify:track:0EUNW9VyHpPPYwENunwz3A,0EUNW9VyHpPPYwENunwz3A,7KnaZr690xW0sCihF9Z8oP,https://i.scdn.co/image/ab67616d0000b273ec0d38...,None
25254,spotify:track:0SNmr8CLffr3khjrX3y4QU,0SNmr8CLffr3khjrX3y4QU,2p4FqHnazRucYQHyDCdBrJ,https://i.scdn.co/image/ab67616d0000b273a093b8...,canadian pop punk
1061,spotify:track:1mJxXKuqwo0TBwf13vBUKH,1mJxXKuqwo0TBwf13vBUKH,0OcclcP5o8VKH2TRqSY2A7,https://i.scdn.co/image/ab67616d0000b273dae458...,canadian classical
32375,spotify:track:6xWXGHp0duQcbJEFOr6qiD,6xWXGHp0duQcbJEFOr6qiD,5zUSfxfP1NETZiaWt0Ui0a,https://i.scdn.co/image/ab67616d0000b273ff095b...,lo-fi beats
2424,spotify:track:5rQQaAUU86ulQ4uZIyS1hp,5rQQaAUU86ulQ4uZIyS1hp,1qqdO7xMptucPDMopsOdkr,https://i.scdn.co/image/ab67616d0000b273cc2ca6...,anthem emo


*Quick note:* I am using `sample` instead of `head` or `tail` just to make the results more interesting when taking a look at the dataset (and avoid judgement - or not).

In [54]:
tracks_history_enriched = tracks_history.merge(
    df_enriched,
    how='left',
    on='spotify_track_uri'
)

tracks_history_enriched.sample(10)

,timestamp_play,platform,ms_played,track_name,artist_name,album_name,spotify_track_uri,reason_start,reason_end,track_id,artist_id,album_cover_uri,artist_genre
185950,2018-01-03T20:21:46Z,"Android OS 7.0 API 24 (motorola, Moto G (4))",3585,All Of The Lights,Kanye West,My Beautiful Dark Twisted Fantasy,spotify:track:1XvHiCTd0MDb16S8bQNl74,trackdone,NaN,1XvHiCTd0MDb16S8bQNl74,5K4W6rqBFWDnAN6FQUkS6x,https://i.scdn.co/image/ab67616d0000b273019f5c...,chicago rap
7246,2022-05-19T19:08:16Z,OS X 12.3.1 [arm 2],164552,Ties That Bind,Alter Bridge,Blackbird,spotify:track:3DjAzhKrzyEKxsSYSoKolW,trackdone,remote,3DjAzhKrzyEKxsSYSoKolW,4DWX7u8BV0vZIQSpJQQDWU,https://i.scdn.co/image/ab67616d0000b273d73fa7...,alternative metal
164527,2020-07-20T18:45:37Z,OS X 10.15.6 [x86 8],99853,Quem Matou o Bozo?,Cueio Limão,Quem Matou o Bozo?,spotify:track:4lT8oTAmjlG1vTDTk0Uoni,trackdone,trackdone,4lT8oTAmjlG1vTDTk0Uoni,1N8HixtL9LGuyvsVNEwfxN,https://i.scdn.co/image/ab67616d0000b273d29a34...,brazilian hardcore
150473,2021-03-14T04:56:15Z,Windows 10 (10.0.19042; x64; AppX),194181,Fearless Pt. II,Lost Sky,Fearless Pt. II,spotify:track:4VZH6OfxPy1nNE7KbtVKyK,trackdone,trackdone,4VZH6OfxPy1nNE7KbtVKyK,157L8iTHgbdrKVxdQEXluh,https://i.scdn.co/image/ab67616d0000b2733be407...,gaming edm
36708,2017-03-09T18:54:37Z,Windows 10 (10.0.14393; x64),201026,In Ashes They Shall Reap,Hatebreed,Hatebreed,spotify:track:30BcmKXRKTCwIGFF0gGfOx,trackdone,NaN,30BcmKXRKTCwIGFF0gGfOx,17Mb968quDHpjCkIyq30QV,https://i.scdn.co/image/ab67616d0000b27300b364...,alternative metal
81633,2020-04-12T03:33:41Z,Windows 10 (10.0.19041; x64; AppX),192120,Drag Me Down,One Direction,Made In The A.M.,spotify:track:2K87XMYnUMqLcX3zvtAF4G,trackdone,trackdone,2K87XMYnUMqLcX3zvtAF4G,4AK6F7OLvEQ5QYCBNiQWHq,https://i.scdn.co/image/ab67616d0000b273241e4f...,boy band
133318,2023-11-10T16:15:17Z,android,233478,Locked out of Heaven,Bruno Mars,Unorthodox Jukebox,spotify:track:3w3y8KPTfNeOKPiqUTakBh,fwdbtn,trackdone,3w3y8KPTfNeOKPiqUTakBh,0du5cEVh5yTK9QJze8zA0C,https://i.scdn.co/image/ab67616d0000b273926f43...,dance pop
71974,2019-10-14T20:32:54Z,OS X 10.15.0 [x86 8],1242,Evil - From Naruto,RMaster,Anime Collection Naruto,spotify:track:3DfHAdEz1ExcpIt8rnn1Gt,clickrow,endplay,3DfHAdEz1ExcpIt8rnn1Gt,4EYP8xAKjfqjJ2bFWX7qLa,https://i.scdn.co/image/ab67616d0000b273caaede...,anime piano
16402,2022-10-23T23:27:55Z,android,61186,Colo de Menina,Rastapé,Fale Comigo,spotify:track:5aGhtbviqUqjtno5vcUmCN,trackdone,logout,5aGhtbviqUqjtno5vcUmCN,4gocL2mlvwlI1G6TAYwQYx,https://i.scdn.co/image/ab67616d0000b2731adade...,brazilian reggae
102903,2018-11-24T04:31:51Z,"Android OS 7.0 API 24 (motorola, Moto G (4))",151036,Helena,My Chemical Romance,Three Cheers for Sweet Revenge,spotify:track:5dTHtzHFPyi8TlTtzoz1J9,clickrow,unexpected-exit-while-paused,5dTHtzHFPyi8TlTtzoz1J9,7FBcuc1gsnv6Y1nwFtNRCb,https://i.scdn.co/image/ab67616d0000b273cab7ae...,emo


Now, my last concern is about the `platform` field available on the dataset. As I am making an overview analysis on my musical taste, I am not interested on know which version of Android or Windows I was using when I was listening to certain track. I just want to know if it was on a PC, my phone or any other device, in general terms.

So, now it's time to take care of this information. First I created a mapping to normalize the names, and then applied it to the `platform` field, creating the `simplified_platform` column. Then, I dropped the original `platform` column and renamed the new column as platform, just to simplify the dataset (confusing?).

*Note:* also, as a matter of Privacy and Security, I dropped the original `platform` column from the dataset, as it will be available publicly on Tableau Public. Not that anyone is interested in my devices and their versions, but I prefer not taking the risk.

In [55]:
tracks_history_enriched.groupby('platform')['spotify_track_uri'].count().sort_values(ascending=False)

platform
Android OS 7.0 API 24 (motorola, Moto G (4))                 19275
osx                                                          17941
Android OS 9 API 28 (Xiaomi, Redmi Note 5)                   16788
Android OS 11 API 30 (Xiaomi, M2101K6G)                      13139
Linux [x86-64 0]                                             10031
                                                             ...  
web_player windows 10;chrome 74.0.3729.169;desktop               1
web_player windows 10;chrome 75.0.3770.90;desktop                1
web_player windows 10;chrome 88.0.4324.104;desktop               1
web_player windows 10;chrome 89.0.4389.90;desktop                1
web_player osx 10.15.5;microsoft edge 83.0.478.58;desktop        1
Name: spotify_track_uri, Length: 103, dtype: int64

In [56]:
platform_mapping = {
    'android': 'Android',
    'osx': 'macOS',
    'os x': 'macOS',
    'windows': 'Windows',
    'web_player': 'Web Browser',
    'linux': 'Linux',
    'ps4': 'PS4',
    'xbox': 'Xbox',
    'tv': 'Smart TV',
    'cast': 'Cast'
}

# Function to simplify platform names
def simplify_platform(platform):
    for key, value in platform_mapping.items():
        if key in platform.lower():
            return value

    return 'Others'

# Apply the simplify_platform function to the "platform" column
tracks_history_enriched['simplified_platform'] = tracks_history_enriched['platform'].apply(simplify_platform)

# Removing the complete platform info for security reasons
tracks_history_enriched.drop(columns=['platform'], inplace=True)
tracks_history_enriched.rename(columns={'simplified_platform': 'platform'}, inplace=True)

tracks_history_enriched.groupby('platform')['track_id'].count().sort_values(ascending=False)

platform
Android        71017
macOS          67017
Windows        38018
Linux          10046
PS4             1484
Cast             909
Xbox             544
Others           347
Smart TV         291
Web Browser      107
Name: track_id, dtype: int64

In [6]:
len(tracks_history_enriched)

189783

In [9]:
tracks_history_enriched = pd.read_csv('consolidated_spotify_data_2024_enriched.csv')
unique_album_covers = tracks_history_enriched.groupby(['artist_name', 'album_name'], as_index=False)[['album_cover_uri']].first()

tracks_history_enriched = tracks_history_enriched.merge(unique_album_covers, how='left', on=['artist_name', 'album_name'], suffixes=('_1', '_2'))
tracks_history_enriched.drop(columns='album_cover_uri_1', inplace=True)
tracks_history_enriched.rename(columns={'album_cover_uri_2': 'album_cover_uri'}, inplace=True)

tracks_history_enriched.head()

,timestamp_play,ms_played,track_name,artist_name,album_name,spotify_track_uri,reason_start,reason_end,track_id,artist_id,artist_genre,platform,album_cover_uri
0,2022-01-22T12:59:36Z,2567,GALOPA,PEDRO SAMPAIO,GALOPA,spotify:track:2wG1R0uDFwyobcWzVssC1J,clickrow,endplay,2wG1R0uDFwyobcWzVssC1J,5wbf52LA6kcaboHSN6NEF1,funk carioca,Android,https://i.scdn.co/image/ab67616d0000b27380dadd...
1,2022-01-22T13:00:24Z,47653,A QUEDA,Gloria Groove,A QUEDA,spotify:track:2s9BO8c0co0PmgBiUoTT17,clickrow,endplay,2s9BO8c0co0PmgBiUoTT17,7rXMvXRnWHaSwnVvPeUUfw,funk carioca,Android,https://i.scdn.co/image/ab67616d0000b273ea1781...
2,2022-01-22T13:04:16Z,218223,Convite de Casamento,Luan Santana,Confraternização Família Santana 2,spotify:track:3BDnZJC9yaj01jtqpyDYzG,clickrow,endplay,3BDnZJC9yaj01jtqpyDYzG,3qvcCP2J0fWi0m0uQDUf6r,arrocha,Android,https://i.scdn.co/image/ab67616d0000b273e70918...
3,2022-01-22T13:05:17Z,61025,Acordando o Prédio,Luan Santana,Acordando o Prédio,spotify:track:40bK2uosUmAS92c17n98xd,clickrow,endplay,40bK2uosUmAS92c17n98xd,3qvcCP2J0fWi0m0uQDUf6r,arrocha,Android,https://i.scdn.co/image/ab67616d0000b273d393a0...
4,2022-01-22T13:05:41Z,23375,Juntos,Paula Fernandes,Juntos,spotify:track:2PIWldP9oUhb9iqe3EJh1e,clickrow,endplay,2PIWldP9oUhb9iqe3EJh1e,1nca3OA1kKCpP6aPJcBL92,sertanejo,Android,https://i.scdn.co/image/ab67616d0000b2733c7178...


Objectives accomplished, now it is time to export the enriched dataset.

In [10]:
tracks_history_enriched.to_csv('consolidated_spotify_data_2024_enriched.csv', index=False)
print('File saved!')

File saved!
